In [1]:
import argparse
import datetime as DT

from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

import re
import itertools

In [2]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
DISCOVERY_URI = ('https://analyticsreporting.googleapis.com/$discovery/rest')
CLIENT_SECRETS_PATH = 'client_secrets.json' # Path to client_secrets.json file.
VIEW_ID = '157208265'

In [3]:
def initialize_analyticsreporting():
    # Set up a Flow object to be used if we need to authenticate.
    flow = client.flow_from_clientsecrets(
                    CLIENT_SECRETS_PATH, scope=SCOPES,
                    message=tools.message_if_missing(CLIENT_SECRETS_PATH))

    # Dummy command line args
    parser = argparse.ArgumentParser(
                    formatter_class=argparse.RawDescriptionHelpFormatter,
                    parents=[tools.argparser])
    flags = parser.parse_args([])

    # Prepare credentials, and authorize HTTP object with them.
    # If the credentials don't exist or are invalid run through the native client
    # flow. The Storage object will ensure that if successful the good
    # credentials will get written back to a file.
    storage = file.Storage('analyticsreporting.dat')
    credentials = storage.get()
    if credentials is None or credentials.invalid:
        credentials = tools.run_flow(flow, storage, flags)
    http = credentials.authorize(http=httplib2.Http())

    # Build the service object.
    analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI)

    return analytics

In [85]:
def get_report(analytics, dimensions=[], metrics=[], filters=[]):
    # Use the Analytics Service Object to query the Analytics Reporting API V4.
    return analytics.reports().batchGet(
        body={
        'reportRequests': [
            {
                'viewId': VIEW_ID,
                'dateRanges': [{'startDate': startDate, 'endDate': endDate}],
                'dimensions': dimensions,
                'metrics': metrics,
                'dimensionFilterClauses': filters
            }]
        }
    ).execute()

In [86]:
def print_response(response, funnel):
  """Parses and prints the Analytics Reporting API V4 response"""

  for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    rows = report.get('data', {}).get('rows', [])

    for row in rows:
      dimensions = row.get('dimensions', [])
      dateRangeValues = row.get('metrics', [])

      for header, dimension in zip(dimensionHeaders, dimensions):
        print (header + ': ' + dimension)

      for i, values in enumerate(dateRangeValues):
        for metricHeader, value in zip(metricHeaders, values.get('values')):
          print (funnel + ': ' + value)

In [91]:
startDate = 'yesterday'
endDate = startDate
analytics = initialize_analyticsreporting()

def incoming_user():
    dimensions = []
    metrics = [{'expression': 'ga:uniquePageviews'}]
    filters = [
        {'filters': [{'dimensionName': 'ga:pagePath', 'expressions': '/core/.+'}]}
    ]
    response = get_report(analytics, dimensions, metrics, filters)
    print_response(response, 'incoming user')
    
incoming_user()

def interacting_user():
    dimensions = []
    metrics = [{'expression': 'ga:uniqueEvents'}]
    filters = [
        {'filters': [{'dimensionName': 'ga:eventCategory', 'expressions': 'scroll depth'}]},
        {'filters': [{'dimensionName': 'ga:eventLabel', 'expressions': '25'}]}
    ]
    response = get_report(analytics, dimensions, metrics, filters)
    print_response(response, 'interacting user')
    filters = [
        {'filters': [{'dimensionName': 'ga:eventLabel', 'expressions': '/core/.+'}]},
        {'filters': [{'dimensionName': 'ga:pagePath', 'expressions': '/core/.+'}]}
    ]
    response = get_report(analytics, dimensions, metrics, filters)
    print_response(response, 'heavily interacting user')

interacting_user()

def interested_user():
    dimensions = []
    metrics = [{'expression': 'ga:uniqueEvents'}]
    filters = [
        {'filters': [{'dimensionName': 'ga:eventCategory', 'expressions': 'showcase'}]},
        {'filters': [{'dimensionName': 'ga:eventAction', 'expressions': 'click'}]},
        {'filters': [{'dimensionName': 'ga:eventLabel', 'expressions': 'premium-data-offer'}]}
    ]
    response = get_report(analytics, dimensions, metrics, filters)
    print_response(response, 'interested user')
    
interested_user()
    

clicked on premium data banner links: 1
